In [37]:
# 必要なモジュールのインストール
# !pip install gspread
# !pip install google-auth

## データの取得

In [38]:
import gspread
from google.oauth2.service_account import Credentials

In [39]:
scopes = [
    'https://www.googleapis.com/auth/spreadsheets',
    'https://www.googleapis.com/auth/drive'
]

credentials = Credentials.from_service_account_file(
    'api_key/google_secret.json',
    scopes=scopes
)

gc = gspread.authorize(credentials)

In [40]:
%load_ext autoreload
%autoreload 2
from api_key import google_key

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [41]:
# スプレッドシートの指定
# シートのキーはスプレッドシートのアドレスから取得可能
SP_SHEET = 'demo'

In [42]:
sh = gc.open_by_key(google_key.SP_SHEET_KEY)

In [43]:
worksheet = sh.worksheet(SP_SHEET)

In [44]:
data = worksheet.get_all_values()
data

[['', '', '', '', ''],
 ['', '社員ID', '氏名', '年齢', '所属'],
 ['', '65', '佐藤', '37', '総務部'],
 ['', '40', '鈴木', '38', '人事部'],
 ['', '31', '岩橋', '42', '経理部'],
 ['', '84', '小泉', '39', '技術部'],
 ['', '74', '今西', '23', '総務部'],
 ['', '36', '斎藤', '43', '企画開発部'],
 ['', '65', '高橋', '24', '技術部'],
 ['', '39', '中村', '48', '技術部'],
 ['', '91', '吉田', '38', '企画開発部'],
 ['', '64', '前田', '24', '開発部']]

In [45]:
import pandas as pd

In [46]:
df = pd.DataFrame(data[2:], columns=data[1])
df

,,社員ID,氏名,年齢,所属
0,,65,佐藤,37,総務部
1,,40,鈴木,38,人事部
2,,31,岩橋,42,経理部
3,,84,小泉,39,技術部
4,,74,今西,23,総務部
5,,36,斎藤,43,企画開発部
6,,65,高橋,24,技術部
7,,39,中村,48,技術部
8,,91,吉田,38,企画開発部
9,,64,前田,24,開発部


In [47]:
df = df.drop(df.columns[0], axis=1)

In [48]:
df

,社員ID,氏名,年齢,所属
0,65,佐藤,37,総務部
1,40,鈴木,38,人事部
2,31,岩橋,42,経理部
3,84,小泉,39,技術部
4,74,今西,23,総務部
5,36,斎藤,43,企画開発部
6,65,高橋,24,技術部
7,39,中村,48,技術部
8,91,吉田,38,企画開発部
9,64,前田,24,開発部


## データの集計

所属ごとの年齢を集計する

In [49]:
df.dtypes

社員ID    object
氏名      object
年齢      object
所属      object
dtype: object

In [50]:
df = df.astype({'年齢': int, '社員ID': int})

In [51]:
df.dtypes

社員ID     int64
氏名      object
年齢       int64
所属      object
dtype: object

In [52]:
pvt_table = df.pivot_table(index=['所属'], values=['年齢'], aggfunc='mean')
pvt_table

,年齢
所属,
人事部,38.0
企画開発部,40.5
技術部,37.0
経理部,42.0
総務部,30.0
開発部,24.0


In [53]:
pvt_table = pvt_table['年齢'].round()
pvt_table

所属
人事部      38.0
企画開発部    40.0
技術部      37.0
経理部      42.0
総務部      30.0
開発部      24.0
Name: 年齢, dtype: float64

## ワークシートの作成

In [54]:
new_worksheet = sh.add_worksheet(title='new', rows=100, cols=10)

In [56]:
# データフレームをスプレッドシートに書くためのライブラリ
# !pip install gspread-dataframe

In [57]:
from gspread_dataframe import set_with_dataframe

In [58]:
start_row = 2
start_col = 2

In [60]:
set_with_dataframe(new_worksheet, pvt_table.reset_index(), row=start_row, col=start_col)

In [59]:
# reset_indexをしないとindexの部分が書かれない
pvt_table.reset_index()

,所属,年齢
0,人事部,38.0
1,企画開発部,40.0
2,技術部,37.0
3,経理部,42.0
4,総務部,30.0
5,開発部,24.0


## シートの書式設定

In [61]:
header_range = 'B2:C2'
index_range = 'B3:B8'
value_range = 'C3:C8'

In [63]:
# スプレッドシートの整形を行うライブラリ
# !pip install gspread-formatting

In [64]:
from gspread_formatting import *

In [66]:
header_fmt = cellFormat(
    backgroundColor=color(38/255, 166/255, 154/255),
    textFormat=textFormat(bold=True, foregroundColor=color(255/255, 255/255, 255/255)),
    horizontalAlignment='CENTER'
)

In [67]:
format_cell_range(new_worksheet, header_range, header_fmt)

{'spreadsheetId': '1VHe1aPV3AVJziSGKWe_hrQJP7eUQxvywEfAsCB_2aF8',
 'replies': [{}]}

In [68]:
border = Border('SOLID', Color(0, 0, 0, 0))

In [69]:
fmt = CellFormat(borders=Borders(top=border, bottom=border, left=border, right=border))
format_cell_range(new_worksheet, header_range, fmt)

{'spreadsheetId': '1VHe1aPV3AVJziSGKWe_hrQJP7eUQxvywEfAsCB_2aF8',
 'replies': [{}]}

In [70]:
format_cell_range(new_worksheet, index_range, fmt)
format_cell_range(new_worksheet, value_range, fmt)

{'spreadsheetId': '1VHe1aPV3AVJziSGKWe_hrQJP7eUQxvywEfAsCB_2aF8',
 'replies': [{}]}